In [1]:
import sys
from pathlib import Path
import torch
import matplotlib.pyplot as plt
from IPython.display import Audio

ROOT_PATH=Path("..").resolve().absolute()
DEVICE=torch.device("cpu")#
sys.path.append(str(ROOT_PATH))

%load_ext autoreload
%autoreload 2
from evaluation.evaluate_separation import evaluate_data, read_ablation_results


In [3]:
path="contextual_dirac_slakh_resample"
read_ablation_results(f"../separations/{path}", f"../separations/{path}")

  0%|                                                                                                                                                  | 0/1 [00:00<?, ?it/s]


AssertionError: <class 'list'>

In [64]:
for path in Path("../logs/experiments_results/").glob("*slack*Crawson*"):
    print("\n---------------------------------")
    sampler, s_churn = path.name.split("-")
    print(f"{sampler=}, {s_churn=}\n")
    evaluate_data(path)


---------------------------------
sampler='slack_sampler=CrawsonEulerExtractor', s_churn='s_churn=1.0'

SI-SNR 1:			 14.703672409057617
SI-SNR 2:			 16.606136322021484
sep1:				 16.052534103393555
sep2:				 16.99407196044922
m - sep1:			 15.730925559997559
m - sep2:			 17.016225814819336
norm 1:			 16.052534103393555
norm 2:			 16.99407196044922
m - sep1 and norm:		 15.730925559997559
m - sep2 and norm:		 17.016225814819336
norm and cons 1:		 16.308622360229492
norm and cons 2:		 17.555713653564453
m - sep1 and norm and cons: 15.831021308898926
m - sep1 and norm and cons: 17.169841766357422

SDR mean:			 16.52330207824707

---------------------------------
sampler='slack_sampler=CrawsonHeunExtractor', s_churn='s_churn=40.0'

SI-SNR 1:			 17.385263442993164
SI-SNR 2:			 19.11650848388672
sep1:				 18.36622428894043
sep2:				 18.997896194458008
m - sep1:			 17.92633628845215
m - sep2:			 19.35613441467285
norm 1:			 18.36622428894043
norm 2:			 18.997896194458008
m - sep1 and norm:		 17

In [65]:
for path in Path("../logs/experiments_results/").glob("*slack*ADPM*"):
    print("\n---------------------------------")
    sampler = path.name.split("-")
    print(f"{sampler=}\n")
    evaluate_data(path)


---------------------------------
sampler=['slack_sampler=ADPM2Extractor']

SI-SNR 1:			 17.306184768676758
SI-SNR 2:			 19.575225830078125
sep1:				 18.172866821289062
sep2:				 19.516815185546875
m - sep1:			 18.29353141784668
m - sep2:			 19.33782196044922
norm 1:			 18.172866821289062
norm 2:			 19.516815185546875
m - sep1 and norm:		 18.29353141784668
m - sep2 and norm:		 19.33782196044922
norm and cons 1:		 18.95686149597168
norm and cons 2:		 20.05926513671875
m - sep1 and norm and cons: 19.131887435913086
m - sep1 and norm and cons: 20.180545806884766

SDR mean:			 18.8448429107666

---------------------------------
sampler=['slack_sampler=ADPM2Separator']

SI-SNR 1:			 7.145374774932861
SI-SNR 2:			 10.345344543457031
sep1:				 7.5242133140563965
sep2:				 10.519533157348633
m - sep1:			 7.520732879638672
m - sep2:			 10.522429466247559
norm 1:			 7.5242133140563965
norm 2:			 10.519533157348633
m - sep1 and norm:		 7.520732879638672
m - sep2 and norm:		 10.522429466247559
no

In [6]:
for path in Path("grid-search-schedule/").glob("*"):
    print("\n---------------------------------")
    smin, smax, rho = path.name.split("-")
    print(f"sigma-min={smin} sigma-max={smax}, rho={rho}\n")
    results = evaluate_data(path)
    print(results["sdr"])
    


---------------------------------
sigma-min=0.0001 sigma-max=20.0, rho=7.0

3.6207435131073

---------------------------------
sigma-min=0.1 sigma-max=1.0, rho=5.0

0.7884336113929749

---------------------------------
sigma-min=0.0001 sigma-max=1.0, rho=7.0

2.2821261882781982

---------------------------------
sigma-min=0.1 sigma-max=20.0, rho=5.0

1.1403182744979858

---------------------------------
sigma-min=0.05 sigma-max=1.0, rho=5.0

1.603183388710022

---------------------------------
sigma-min=0.1 sigma-max=5.0, rho=5.0

1.4022505283355713

---------------------------------
sigma-min=0.05 sigma-max=1.0, rho=9.0

1.6622364521026611

---------------------------------
sigma-min=0.0001 sigma-max=5.0, rho=5.0

3.4981727600097656

---------------------------------
sigma-min=0.1 sigma-max=20.0, rho=9.0

1.1558301448822021

---------------------------------
sigma-min=0.0001 sigma-max=5.0, rho=9.0

3.479304075241089

---------------------------------
sigma-min=0.1 sigma-max=5.0, rho=

In [25]:
#!tar czf "sep.tar.xz"  "separations-musdb-irene-2/"

from pathlib import Path
import torchaudio
import tqdm
from main.separation import enforce_mixture_consistency

def load_separation_data(separation_path):
    separation_folder = Path(separation_path)
    seps1, seps2, oris1, oris2, ms = [], [], [], [], []
    chunks = []
    
    for chunk_folder in tqdm.tqdm(list(separation_folder.glob("*"))):
        original_tracks = [torchaudio.load(ori) for ori in sorted(list(chunk_folder.glob("ori*.wav")))]
        separated_tracks = [torchaudio.load(sep) for sep in sorted(list(chunk_folder.glob("sep*.wav")))]

        original_tracks, sample_rates_ori = zip(*original_tracks)
        separated_tracks, sample_rates_sep = zip(*separated_tracks)

        assert len({*sample_rates_ori, *sample_rates_sep}) == 1
        sample_rate = sample_rates_ori[0]

        assert len(original_tracks) == len(separated_tracks)
        assert len(original_tracks) == 2
        ori1, ori2 = original_tracks
        sep1, sep2 = separated_tracks

        m = ori1 + ori2

        #if torch.amax(torch.abs(ori1)) < 1e-3 or torch.amax(torch.abs(ori2)) < 1e-3:
        #    continue
        
        chunks.append(int(chunk_folder.name))
        seps1.append(sep1)
        seps2.append(sep2)
        oris1.append(ori1)
        oris2.append(ori2)
        ms.append(m)

    seps1 = torch.stack(seps1, dim=0)
    seps2 = torch.stack(seps2, dim=0)
    seps = torch.stack([seps1, seps2], dim=1)

    oris1 = torch.stack(oris1, dim=0)
    oris2 = torch.stack(oris2, dim=0)
    oris = torch.stack([oris1, oris2], dim=1)

    ms = torch.stack(ms, dim=0)
    return [seps1,seps2], [oris1,oris2], ms, chunks

def sdr(preds, targets):
    preds = preds.cpu()
    targets = targets.cpu()

    signal = torch.norm(targets) ** 2
    e_error = torch.norm(preds - targets) ** 2
    return 10 * torch.log10( signal/e_error )


(seps1,seps2),(oris1, oris2), ms, chunks = load_separation_data("separations-musdb-dpm2")

tmp = list(zip(chunks, seps1,seps2,oris1,oris2, ms))
tmp = sorted(tmp, key=lambda x:x[0])

bass_sdrs, drums_sdrs = [], []


def is_non_zero(source_waveforms):
  """Return shape (source,) weights for signals that are nonzero."""
  source_norms = torch.sqrt(torch.mean(source_waveforms**2, axis=-1))
  return source_norms > 1e-8

for (chunk,s1,s2,o1,o2,m) in tmp:
    #if torch.amax(torch.abs(o1)) < 1e-3 or torch.amax(torch.abs(o2)) < 1e-3:
    #    continue
    
    if not is_non_zero(o1) or not is_non_zero(o2):
        continue
        
    s1,s2 = enforce_mixture_consistency(m, torch.stack([s1.view(1,1,-1), s2.view(1,1,-1)], dim=1))
    print(f"[{chunk:02d}]: \t({torch.amax(torch.abs(o1)):.4f}) \t({torch.amax(torch.abs(o2)):.4f})\t bass={sdr(s1,o1):.3f}\t drums={sdr(s2,o2):.4f}")
    bass_sdrs.append(sdr(s1,o1))
    drums_sdrs.append(sdr(s2,o2))

sdr1 = sum(bass_sdrs)/len(bass_sdrs)
sdr2 = sum(drums_sdrs)/len(drums_sdrs)
print(sdr1, sdr2)
print((sdr1+sdr2)*0.5)


100%|██████████| 48/48 [00:00<00:00, 147.30it/s]


[00]: 	(0.7027) 	(1.0000)	 bass=-1.833	 drums=6.1318
[01]: 	(0.6571) 	(1.0000)	 bass=4.425	 drums=2.1312
[02]: 	(0.6682) 	(1.0000)	 bass=1.939	 drums=3.7263
[03]: 	(0.5316) 	(1.0000)	 bass=7.625	 drums=2.7491
[04]: 	(0.4069) 	(1.0000)	 bass=3.983	 drums=4.8827
[05]: 	(0.1375) 	(0.4310)	 bass=0.931	 drums=6.1348
[06]: 	(0.4774) 	(1.0000)	 bass=0.663	 drums=6.8117
[07]: 	(0.5299) 	(1.0000)	 bass=3.077	 drums=6.5311
[08]: 	(0.3761) 	(0.8795)	 bass=1.650	 drums=5.5166
[09]: 	(0.3382) 	(1.0000)	 bass=-1.004	 drums=7.4564
[10]: 	(0.2447) 	(1.0000)	 bass=-2.237	 drums=5.7632
[11]: 	(0.7815) 	(1.0000)	 bass=-1.355	 drums=5.9645
[12]: 	(0.3378) 	(0.9728)	 bass=-4.939	 drums=6.4330
[14]: 	(0.4099) 	(1.0000)	 bass=0.986	 drums=4.3455
[15]: 	(0.1440) 	(0.6007)	 bass=-20.446	 drums=7.9618
[16]: 	(0.4570) 	(0.9467)	 bass=8.674	 drums=7.6329
[17]: 	(0.4325) 	(0.9502)	 bass=-0.954	 drums=5.7250
[18]: 	(0.6845) 	(1.0000)	 bass=0.459	 drums=6.0074
[19]: 	(0.4694) 	(0.8108)	 bass=7.217	 drums=7.2800
[20]

In [2]:
evaluate_data("separations-musdb-dpm2")

100%|██████████| 48/48 [00:00<00:00, 124.57it/s]


SI-SDR piano:  0.47045114636421204
SI-SDR basso:  3.94543194770813
SI-SDRi piano:  3.2123315632343292
SI-SDRi basso:  1.348285436630249

SI-SNR piano:  0.471749871969223
SI-SNR basso:  3.9458775520324707
SI-SNRi piano:  3.21295365691185
SI-SNRi basso:  1.3492193222045898

SDR mean:  3.3860299489753944
3.6767312666719043e-10
SDRi mean:  3.3860299486077214


## Compute separation metrics

In [3]:
import torchmetrics.functional.audio as tma
from pathlib import Path
import torchaudio
import torch
import tqdm
from main.separation import enforce_mixture_consistency


def si_snr(preds, target):
    return tma.scale_invariant_signal_noise_ratio(preds=preds.cpu(), target=target.cpu()).mean().item()

def si_sdr(preds, target):
    return tma.scale_invariant_signal_distortion_ratio(preds=preds.cpu(), target=target.cpu()).mean().item()

def sdr(preds, target):
    return tma.signal_distortion_ratio(preds=preds.cpu(), target=target.cpu()).mean().item()

In [4]:
separation_folder = Path("separations")

seps1, seps2, oris1, oris2, ms = [], [], [], [], []
for file in tqdm.autonotebook.tqdm(list(separation_folder.glob("*"))):
    ori1, _ = torchaudio.load(file / "ori1.wav")
    ori2, _ = torchaudio.load(file / "ori2.wav")
    sep1, _ = torchaudio.load(file / "sep1.wav")
    sep2, _ = torchaudio.load(file / "sep2.wav")
    m = ori1 + ori2
    
    if torch.amax(torch.abs(ori1 - m)) < 1e-5 or torch.amax(torch.abs(ori2 - m)) < 1e-5:
        continue
    
    seps1.append(sep1)
    seps2.append(sep2)
    oris1.append(ori1) 
    oris2.append(ori2)
    ms.append(m)

seps1 = torch.cat(seps1, dim=0)
seps2 = torch.cat(seps2, dim=0)
oris1 = torch.cat(oris1, dim=0)
oris2 = torch.cat(oris2, dim=0)
ms = torch.cat(ms, dim=0)

print("SI-SDR piano: ",si_sdr(seps1, oris1))
print("SI-SDR basso: ",si_sdr(seps2, oris2))
print("SI-SDRi piano: ", si_sdr(seps1, oris1) - si_sdr(ms, oris1))
print("SI-SDRi basso: ", si_sdr(seps2, oris2) - si_sdr(ms, oris2))
print()
print("SI-SNR piano: ", si_snr(seps1, oris1))
print("SI-SNR basso: ", si_snr(seps2, oris2))
print("SI-SNRi piano: ", si_snr(seps1, oris1) - si_snr(ms, oris1))
print("SI-SNRi basso: ", si_snr(seps2, oris2) - si_snr(ms, oris2))
print()
print("SDR piano: ",sdr(seps1, oris1))
print("SDR basso: ",sdr(seps2, oris2))
print("SDRi piano: ",sdr(seps1, oris1) - sdr(ms, oris1) )
print("SDRi basso: ",sdr(seps2, oris2) - sdr(ms, oris2) )

#print(f"SI-SNRi (1): {si_snri_1/count}")
#print(f"SI-SNRi (2): {si_snri_2/count}")
#    print(f"SI-SNR (mix): {si_snr(sep1+sep2, m)}\n")
    
    
    


0it [00:00, ?it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [15]:
from main.separation import least_squares_normalization

sampling_rate = 22050
for path in Path("../logs/experiments_results/").glob("sampler=Crawson*/"):
    print("\n---------------------------------")
    sampler, s_churn = path.name.split("-")
    print(f"sampler={sampler} sigma-s_churn={s_churn}\n")
    _, seps, oris = evaluate_data(path)
    cons_seps = []
    cons_seps_n = []
    cons_seps_m = []
    for i, (sep, ori) in enumerate(zip(seps, oris)):
        mixture = ori.sum(dim=0, keepdims=True)
        sepa1m, sepa2m = enforce_mixture_consistency(mixture, sep.unsqueeze(0))
        sepa1n, sepa2n = least_squares_normalization(sep[0], sep[1], mixture)
        sepa1, sepa2 = enforce_mixture_consistency(mixture, torch.stack([sepa1n, sepa2n], dim=1).unsqueeze(0))
        sepam = torch.stack([sepa1m, sepa2m], dim=1)
        sepan = torch.stack([sepa1n, sepa2n], dim=1)
        sepa = torch.stack([sepa1, sepa2], dim=1)
        cons_seps_m.append(sepam)
        cons_seps_n.append(sepan)
        cons_seps.append(sepa)
        if i == 1:
            break
        #if i==0:
        #    display(Audio(sepa1.reshape(1,-1), rate = sampling_rate))
        #    display(Audio(sepa2.reshape(1,-1), rate = sampling_rate))
        #    display(Audio(ori[0].reshape(1,-1), rate = sampling_rate))
        #    display(Audio(ori[1].reshape(1,-1), rate = sampling_rate))
    cons_seps_m = torch.stack(cons_seps_m).squeeze(1)
    cons_seps_n = torch.stack(cons_seps_n).squeeze(1)
    cons_seps = torch.stack(cons_seps).squeeze(1)
    print(f"SDR mixture cons: {sdr(cons_seps, oris)}")
    print(f"SDR normalized: {sdr(cons_seps_n, oris)}")
    print(f"SDR mixture cons and normalized: {sdr(cons_seps, oris)}")


---------------------------------
sampler=sampler=CrawsonEulerSeparatorBASIS sigma-s_churn=s_churn=20.0

SI-SNR 1:  1.4338979721069336
SI-SNR 2:  2.268383741378784
SI-SNRi 1:  1.5986348688602448
SI-SNRi 2:  1.907227486371994
SDR mean:  2.2187018394470215


ValueError: not enough values to unpack (expected 2, got 1)

In [81]:
for path in Path("../logs/experiments_results/").glob("sampler=ADPM2*/"):
    print("\n---------------------------------")
    sampler = path.name.split("-")
    print(f"sampler={sampler}\n")
    _, seps, oris = evaluate_data(path)
    cons_seps = []
    for i, (sep, ori) in enumerate(zip(seps, oris)):
        mixture = ori.sum(dim=0, keepdims=True)
        sepa1, sepa2 = enforce_mixture_consistency(mixture, sep.unsqueeze(0))
        sepa = torch.stack([sepa1, sepa2], dim=1)
        cons_seps.append(sepa)
    cons_seps = torch.stack(cons_seps).squeeze(1)
    print(f"SDR mixture cons: {sdr(cons_seps, oris)}")


---------------------------------
sampler=['sampler=ADPM2Extractor']

SI-SNR 1:  0.6253742575645447
SI-SNR 2:  1.4158798456192017
SI-SNRi 1:  0.7901111543178558
SI-SNRi 2:  1.0547235906124115
SDR mean:  1.8196426630020142
SDR mixture cons: 2.5656189918518066

---------------------------------
sampler=['sampler=ADPM2Separator']

SI-SNR 1:  0.8745163679122925
SI-SNR 2:  1.3886007070541382
SI-SNRi 1:  1.0392532646656036
SI-SNRi 2:  1.027444452047348
SDR mean:  1.505374550819397
SDR mixture cons: 1.5054229497909546


# Compute data statistics

In [ ]:
from audio_data_pytorch import WAVDataset
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
from audio_data_pytorch.transforms import Loudness

dataset = WAVDataset("/data/MusDB/data/drums/train")

fn = lambda x:x #Loudness(44100, -20)
wavs_norms = pd.Series([np.linalg.norm(fn(wav)) for wav in tqdm(dataset)])

In [ ]:
wav_norm_normalized = wavs_norms / wavs_norms.mean()
print(wav_norm_normalized.describe())
wav_norm_normalized.hist(bins=30)

In [ ]:
wavs_stds = pd.Series([wav.std().item() for wav in tqdm(dataset)])
wavs_stds.describe()

In [ ]:
wav = dataset[0]#,int(1e6):int(1e6)+ 50].numpy()
wav_diff = np.diff(np.diff(wav))
plt.plot(wav)
plt.plot(wav_diff/np.linalg.norm(wav_diff)*np.linalg.norm(wav))
si_snr(wav[:-2], torch.from_numpy(wav_diff))

In [ ]:
from main.dataset import ChunkedWAVDataset 
from audio_data_pytorch import AllTransform
dataset_val = ChunkedWAVDataset(
    path="/data/MusDB/data/drums/test", 
    max_chunk_size=22050,
    recursive=True,
    transforms=AllTransform(source_rate=44100, target_rate=44100, mono=True),
)

In [ ]:
import scipy
conv1d = scipy.signal.fftconvolve
norm = np.linalg.norm


def compute_cossim(wav:np.ndarray, chunk:np.ndarray):
    chunk_size = chunk.shape[-1]
    chunk_norm = np.linalg.norm(chunk)
    dot_wav = conv1d(wav, np.flip(chunk), mode="same")
    wav_norm = np.sqrt(conv1d(wav**2, np.ones_like(chunk), mode="same"))
    wav_norm[np.isnan(wav_norm)] = 0.0
    
    cossim = np.zeros_like(dot_wav)
    mask = wav_norm != 0
    cossim[mask] = dot_wav[mask] / (wav_norm[mask] * chunk_norm)
    return cossim

chunk = dataset_val[0][0].numpy()
plt.plot(chunk)
plt.show()

cossims = []
for wav in tqdm(dataset):
    wav = wav[0].numpy()
    cossim = compute_cossim(wav, chunk)
    i = cossim.argmax()
    cossims.append( (i, cossim[i]) )
    

#wav = dataset[1][0].numpy()
#chunk = dataset_val[100][0].numpy()

def compute_cossim(wav:np.ndarray, chunk:np.ndarray):
    chunk_size = chunk.shape[-1]
    chunk_norm = np.linalg.norm(chunk)
    dot_wav = conv1d(wav, np.flip(chunk), mode="same")
    wav_norm = np.sqrt(conv1d(wav**2, np.ones_like(chunk), mode="same"))
    wav_norm[np.isnan(wav_norm)] = 0.0
    
    cossim = np.zeros_like(dot_wav)
    mask = wav_norm != 0
    cossim[mask] = dot_wav[mask] / (wav_norm[mask] * chunk_norm)
    return cossim

In [ ]:
a,b = zip(*cossims)
chunk_size = chunk.shape[-1]
wav_idx, chunk_idx, val = max(zip(range(len(a)),a,b),key=lambda x:x[-1])

wav_chunk = dataset[wav_idx][0,chunk_idx-chunk_size//2:chunk_idx+chunk_size//2]

print(wav_idx, chunk_idx, val)

plt.plot(chunk/norm(chunk))
plt.plot(wav_chunk/norm(wav_chunk))
display(Audio(wav_chunk, rate=44100))

np.dot(wav_chunk, chunk)/(norm(wav_chunk)*norm(chunk))

In [ ]:
i= 1000001
chunk_slice = slice(i-chunk_size//2,i+chunk_size//2)
print(wav_norm[i], np.linalg.norm(wav[chunk_slice]))
print(np.dot(wav[chunk_slice], chunk), dot_wav[i])

In [ ]:
display(Audio(chunk, rate=44100))
plt.plot(cossim)

In [ ]:
i = (cossim>0.1).nonzero()[0][1000]#cossim.argmax()
print(i,":",cossim[i])

plt.plot(chunk/chunk_norm)
wav_chunk = wav[i-chunk_size//2:i+chunk_size//2]
plt.plot(wav_chunk/wav_norm[i])
print(wav.shape)
display(Audio(wav_chunk, rate=44100))

print(dot_wav[i])
np.dot(wav_chunk, chunk)/(norm(wav_chunk)*norm(chunk))

In [ ]:
interpol = np.arange(chunk_size)/chunk_size
display(Audio(wav_chunk*interpol + (1-interpol)*chunk, rate=44100))
print(interpol)

### FAD

In [4]:
from frechet_audio_distance import FrechetAudioDistance
from pathlib import Path
ROOT_PATH = Path("..").resolve().absolute()


frechet = FrechetAudioDistance(
    use_pca=False, 
    use_activation=False,
    verbose=False
)
fad_score = frechet.score(ROOT_PATH / "data/dummy/gt", ROOT_PATH / "data/dummy/generated")

Using cache found in /home/irene/.cache/torch/hub/harritaylor_torchvggish_master


[Frechet Audio Distance] exception thrown, Error opening '/home/irene/Documents/audio-diffusion-pytorch-trainer/data/dummy/gt/Track01876.wav': Format not recognised.


In [8]:
import torchaudio
from IPython.display import Audio
import math

sampling_rate = 22050
ROOT_PATH = Path("..").resolve().absolute()

s1, sr1 = torchaudio.load(ROOT_PATH / 'data/Slack/test/drums/Track01876.wav')
s2, sr2 = torchaudio.load(ROOT_PATH / 'data/Slack/test/piano/Track01876.wav')
s3, sr3 = torchaudio.load(ROOT_PATH / 'data/Slack/test/drums/Track01876.wav')
s4, sr4 = torchaudio.load(ROOT_PATH / 'data/Slack/test/piano/Track01876.wav')

s1 = torchaudio.functional.resample(s1, orig_freq=sr1, new_freq=sampling_rate)
s2 = torchaudio.functional.resample(s2, orig_freq=sr1, new_freq=sampling_rate)
s3 = torchaudio.functional.resample(s3, orig_freq=sr1, new_freq=sampling_rate)
s4 = torchaudio.functional.resample(s4, orig_freq=sr1, new_freq=sampling_rate)

start_sample = 100 * sampling_rate
length = 10
length_samples = 2**math.ceil(math.log2(length * sampling_rate))
s1 = s1.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
s2 = s2.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
s3 = s3.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
s4 = s4.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]

m = s1 + s2 + s3 + s4



In [9]:
import torch
torch.save(m, ROOT_PATH / "data/dummy/gt/Track01876.wav")

In [ ]:
def append_dims(x, target_dims):
    """Appends dimensions to the end of a tensor until it has target_dims dimensions."""
    dims_to_append = target_dims - x.ndim
    if dims_to_append < 0:
        raise ValueError(f'input has {x.ndim} dims but target_dims is {target_dims}, which is less')
    return x[(...,) + (None,) * dims_to_append]

def to_d(x, sigma, denoised):
    """Converts a denoiser output to a Karras ODE derivative."""
    return (x - denoised) / append_dims(sigma, x.ndim)

@torch.no_grad()
def log_likelihood(model, x, sigma_min, sigma_max, extra_args=None, atol=1e-4, rtol=1e-4):
    extra_args = {} if extra_args is None else extra_args
    s_in = x.new_ones([x.shape[0]])
    v = torch.randint_like(x, 2) * 2 - 1
    fevals = 0
    def ode_fn(sigma, x):
        nonlocal fevals
        with torch.enable_grad():
            x = x[0].detach().requires_grad_()
            denoised = model(x, sigma * s_in, **extra_args)
            d = to_d(x, sigma, denoised)
            fevals += 1
            grad = torch.autograd.grad((d * v).sum(), x)[0]
            d_ll = (v * grad).flatten(1).sum(1)
        return d.detach(), d_ll
    x_min = x, x.new_zeros([x.shape[0]])
    t = x.new_tensor([sigma_min, sigma_max])
    sol = odeint(ode_fn, x_min, t, atol=atol, rtol=rtol, method='dopri5')
    latent, delta_ll = sol[0][-1], sol[1][-1]
    ll_prior = torch.distributions.Normal(0, sigma_max).log_prob(latent).flatten(1).sum(1)
    return ll_prior + delta_ll, {'fevals': fevals}